In [1]:
%load_ext autoreload
%autoreload 2
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('klue/roberta-large')

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at klue/roberta-large were not used when initializing BertModel: ['roberta.encoder.layer.17.attention.output.LayerNorm.weight', 'roberta.encoder.layer.13.attention.output.dense.weight', 'roberta.encoder.layer.17.output.LayerNorm.bias', 'roberta.encoder.layer.18.attention.output.dense.weight', 'lm_head.layer_norm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.7.output.LayerNorm.weight', 'roberta.encoder.layer.8.output.dense.weight', 'roberta.encoder.layer.14.intermedia

In [11]:
from Fishing.dataset import BaitDataset
config = {
    'max_word_len': 128,
    'bait_path': '/workspace/codes/02_DSBA_Project/bait_news_gen/joonghoon/t5_base/tfidf_content/content_chunking_forward',
    'data_path': '/workspace/codes/02_DSBA_Project/bait_news_gen/data/original',
    'sort': 'News_Direct',
}
dataset = BaitDataset(config, split='train', tokenizer=tokenizer)
bait_title, news_title, bait_file_path = dataset.load_bait_news_info(data_dir=config['data_path'], bait_dir=config['bait_path'], split='train')

Load Data train | News : 100%|██████████| 39996/39996 [00:02<00:00, 14516.25it/s]
Load Data train | Bait : 100%|██████████| 36969/36969 [00:02<00:00, 13155.77it/s]
Load Data train | Bait : 100%|██████████| 39996/39996 [00:03<00:00, 12855.55it/s]
Load Data validation | Bait : 100%|██████████| 39996/39996 [00:03<00:00, 13104.79it/s]
Load Data test | Bait : 100%|██████████| 39996/39996 [00:03<00:00, 12823.02it/s]


In [12]:
len(bait_title), len(news_title)

(39996, 39996)

In [17]:
refs = list(news_title.values())
cands = list(bait_title.values())
from KoBERTScore.KoBERTScore import BERTScore
kobertscore = BERTScore(model_name_or_path='klue/roberta-large')
score = kobertscore(refs, cands)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at klue/roberta-large were not used when initializing BertModel: ['roberta.encoder.layer.17.attention.output.LayerNorm.weight', 'roberta.encoder.layer.13.attention.output.dense.weight', 'roberta.encoder.layer.17.output.LayerNorm.bias', 'roberta.encoder.layer.18.attention.output.dense.weight', 'lm_head.layer_norm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.7.output.LayerNorm.weight', 'roberta.encoder.layer.8.output.dense.weight', 'roberta.encoder.layer.14.intermediate.dense.weight', 'roberta.encoder.layer.8.attention.self.key.bias', 'roberta.encoder.layer.12.attention.output.dense.bias', 'roberta.encoder.layer.3.intermediate.dense.bias', 'roberta.encoder.layer.7.attention.self.key.bias', 'roberta.encoder.

Load klue/roberta-large with 22 layers


Calculating BERTScore: 100%|██████████| 313/313 [01:27<00:00,  3.57it/s]


In [26]:
import torch
index = torch.sort(torch.Tensor(score))[1]
score = torch.sort(torch.Tensor(score))[0]

In [52]:
index_ = index[-15000]
print(f"Score: {score[index_]}")
print(f"Reference: {refs[index_]}")
print(f"Candidate: {cands[index_]}")

Score: 0.9535970091819763
Reference: 삼성디스플레이, 최대 10조원 OLED 투자 '스타트'
Candidate: 삼성디스플레이, 올해 OLED 설비투자 10조원...2년 연속 뭉칫돈
